# Export to ONNX and inference using TensorRT

<div class="alert alert-info">

<b>Note:</b>

Currently, only FP8 delayed scaling is supported for export to ONNX. MXFP8 support is currently under development.

</div>

Transformer Engine (TE) is a library designed primarily for training models in low precision. It is not specifically optimized for inference tasks, so other dedicated solutions should be used. NVIDIA provides numerous [inference tools](https://www.nvidia.com/en-us/solutions/ai/inference/) that enhance the entire inference pipeline. Two prominent NVIDIA inference SDKs are [TensorRT](https://github.com/NVIDIA/TensorRT) and [TensorRT-LLM](https://github.com/NVIDIA/TensorRT-LLM).

This tutorial illustrates how you can export a PyTorch model to ONNX format and subsequently perform inference with TensorRT. This approach is particularly beneficial if your model integrates Transformer Engine layers within more complex architectures. It's important to highlight that for Transformer-based large language models (LLMs), TensorRT-LLM could provide a more optimized inference experience. However, the ONNX-to-TensorRT approach described here may be more suitable for other models, such as diffusion-based architectures or vision transformers.

Let's begin by defining a simple model composed of layers both from Transformer Engine and standard PyTorch:

In [1]:
import torch 
import torch.nn as nn
import transformer_engine as te
import warnings
warnings.filterwarnings("ignore")

B, S, H = 256, 256, 512

class Model(torch.nn.Module):
    def __init__(self, hidden_dim=H, num_non_te_layers=16, num_te_layers=4, num_te_heads=4):
        super(Model, self).__init__()
        self.non_te_part = nn.Sequential(
            *[nn.Sequential(nn.Linear(hidden_dim, hidden_dim), nn.GELU()) for _ in range(num_non_te_layers)]
        )
        self.te_part = nn.Sequential(
            *[te.pytorch.TransformerLayer(hidden_dim, hidden_dim, num_te_heads) for _ in range(num_te_layers)]
        )

    def forward(self, x):
        x = self.non_te_part(x)
        return self.te_part(x)

Let's run some simple inference benchmarks:

In [2]:
from utils import  _measure_time

model = Model().eval().cuda()
inps = (torch.randn([S, B, H], device="cuda"),)
def _inference(fp8_enabled):
     with torch.no_grad(), te.pytorch.fp8_autocast(fp8_enabled=fp8):
        model(*inps)

te_fp32_time = _measure_time(lambda: _inference(fp8_enabled=False))
te_fp8_time = _measure_time(lambda: _inference(fp8_enabled=True))

print(f"Average inference time FP32: {te_fp32_time} ms")
print(f"Average inference time FP8: {te_fp8_time} ms")

Average inference time FP32: 0.05 ms
Average inference time FP8: 0.044 ms


#### Exporting the TE Model to ONNX Format

PyTorch is developing a new [ONNX exporter](https://pytorch.org/docs/stable/onnx.html) built on TorchDynamo and plans to phase out the existing TorchScript exporter. As this feature is currently in active development, we recommend running this process with the latest PyTorch version.


To export a Transformer Engine model into ONNX format, follow these steps:

- Conduct warm-up runs within autocast using the recipe intended for export.
- Encapsulate your export-related code within `te.onnx_export`, ensuring warm-up runs remain outside this wrapper.
- Use the PyTorch Dynamo ONNX exporter by invoking: `torch.onnx.export(..., dynamo=True)`.

In [3]:
from transformer_engine.pytorch.export import te_translation_table

def export(model, fname, inputs, fp8=True):
    with torch.no_grad(), te.fp8_autocast(enabled=fp8):
        # ! IMPORTANT !
        # Transformer Engine models must have warm-up runs
        # before export. The warm-up recipe should match 
        # the actual export recipe.
        model(*inputs)

        # Only dynamo=True mode is supported;
        # dynamo=False is deprecated and unsupported.
        #
        # te_translation_table contains necessary ONNX translations
        # for FP8 quantize/dequantize operators.
        print(f"Exporting {fname}")
        with te.onnx_export(enabled=True):
            torch.onnx.export(
                model,
                inputs,
                fname,
                output_names=["output"],
                dynamo=True,
                custom_translation_table=te_translation_table
            )

# Example usage:
inputs = (your_input_tensor,)  # Replace this with actual input tensors
export(model, "model_fp8.onnx", inputs, fp8=True)
export(model, "model_fp32.onnx", inputs, fp8=False)

Exporting model_fp8.onnx
[torch.onnx] Obtain model graph for `Model([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `Model([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 4 of general pattern rewrite rules.
Exporting model_fp32.onnx
[torch.onnx] Obtain model graph for `Model([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `Model([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 4 of general pattern rewrite rules.


#### Inference with TensorRT

TensorRT is a high-performance deep learning inference optimizer and runtime developed by NVIDIA. It enables optimized deployment of neural network models by maximizing inference throughput and reducing latency on NVIDIA GPUs. TensorRT performs various optimization techniques, including layer fusion, precision calibration, kernel tuning, and memory optimization. 
For detailed information and documentation, refer to the official [TensorRT documentation](https://developer.nvidia.com/tensorrt).

When using TensorRT, ONNX model must first be compiled into a TensorRT engine. This compilation step involves converting the ONNX model into an optimized representation tailored specifically to the target GPU platform. The compiled engine file can then be loaded into applications for rapid and efficient inference execution.

In [4]:
!trtexec --onnx=model_fp32.onnx --saveEngine=model_fp32.engine > output_fp32.log 2>&1
!trtexec --onnx=model_fp8.onnx --saveEngine=model_fp8.engine > output_fp8.log 2>&1

Let's run the benchmarks for inference:

In [5]:
import tensorrt as trt

output_tensor = torch.empty_like(model(*inps))

def load_engine(engine_file_path):
    logger = trt.Logger(trt.Logger.WARNING)
    runtime = trt.Runtime(logger)
    
    with open(engine_file_path, "rb") as f:
        engine_data = f.read()
    
    engine = runtime.deserialize_cuda_engine(engine_data)
    return engine

def benchmark_inference(model_name):
    engine = load_engine(model_name)
    context = engine.create_execution_context()
    stream = torch.cuda.Stream()
    
    for i in range(len(inps)):
        context.set_tensor_address(engine.get_tensor_name(i), inps[i].data_ptr())    
    context.set_tensor_address("output", output_tensor.data_ptr())
    
    def _inference():
        context.execute_async_v3(stream_handle=stream.cuda_stream)
        stream.synchronize()
        
    return _measure_time(_inference)


trt_fp8_time = benchmark_inference("model_fp8.engine")
trt_fp32_time = benchmark_inference("model_fp32.engine")

print(f"Average TE inference time (FP32): {te_fp32_time} ms")
print(f"Average TE inference time (FP8): {te_fp8_time} ms, speedup = {te_fp32_time/te_fp8_time:.2f}x")
print(f"Average TRT inference time (FP32): {trt_fp32_time:.4f} ms, speedup = {te_fp32_time/trt_fp32_time:.2f}x")
print(f"Average TRT inference time (FP8): {trt_fp8_time:.4f} ms, speedup = {te_fp32_time/trt_fp8_time:.2f}x")

Average TE inference time (FP32): 0.05 ms
Average TE inference time (FP8): 0.044 ms, speedup = 1.14x
Average TRT inference time (FP32): 0.0430 ms, speedup = 1.16x
Average TRT inference time (FP8): 0.0330 ms, speedup = 1.52x


We clearly observe performance improvements when using FP8 and the TensorRT inference engine. These improvements may become even more significant with more complex models, as TensorRT will identify additional optimization opportunities.

#### Low Precision Operators in ONNX and TensorRT

The ONNX standard does not currently support all precision types provided by the Transformer Engine. All available ONNX operators are listed on [this website](https://onnx.ai/onnx/operators/). Consequently, TensorRT and the Transformer Engine utilize certain specialized low-precision operators, detailed below.

**TRT_FP8_QUANTIZE**

- **Name**: TRT_FP8_QUANTIZE
- **Domain**: trt
- **Inputs**:
    - `x`: float32 tensor
    - `scale`: float32 scalar
- **Outputs**:
    - `y`: int8 tensor

Produces an int8 tensor that represents the binary encoding of FP8 values.

**TRT_FP8_DEQUANTIZE**

- **Name**: TRT_FP8_DEQUANTIZE
- **Domain**: trt
- **Inputs**:
    - `x`: int8 tensor
    - `scale`: float32 scalar
- **Outputs**:
    - `y`: float32 tensor

Converts FP8-encoded int8 tensor data back into float32 precision.

Since standard ONNX operators do not support certain input and output precision types, a workaround is employed: tensors are dequantized to higher precision (float32) before input into these operators or quantized to lower precision after processing. TensorRT recognizes such quantize-dequantize patterns and replaces them with optimized operations. More details are available in [this section](https://docs.nvidia.com/deeplearning/tensorrt/latest/inference-library/work-quantized-types.html#tensorrt-processing-of-q-dq-networks) of the TensorRT documentation.